### Team Schedule

In [1]:
# Import libraries.
import pandas as pd, numpy as np, os
from pandas import Timestamp
from ipywidgets import interact, fixed, IntSlider, Button
from datetime import date, datetime 
import matplotlib.pyplot as plt
from openpyxl import load_workbook
from Gantt_plot import *
pd.options.display.float_format = '{:,.3f}'.format

In [2]:
xlsm = "Work Schedule.xlsm"

In [3]:
# All sheetnames in workbook.
wb = load_workbook(xlsm, read_only=True, keep_links=False)
sh_names = [w for w in wb.sheetnames if w not in ["Team"]]
wb.close()

data = dict()
for sh_name in sh_names:
    df = pd.read_excel(xlsm, sheet_name=sh_name)
    if sh_name=="schedule":
        # Schedule: ongoing project only
        df = df.drop(columns=["start_date","days"]).fillna(0)
        df = df.loc[df["type"].isin(["Project"]) & 
                    (df["completion"]<1)].reset_index(drop=True)
        data[sh_name] = df.to_dict()
    elif sh_name=="PublicHolidays":
        # Public holidays
        data[sh_name] = np.array(df.drop(columns=["Holiday"]), 
                                 dtype="datetime64[D]").ravel()
    else: data[sh_name] = df.to_dict()

## Master Schedule

In [4]:
def interactive_gantt(df, ref_date, holidays, show_delta, 
                      backward, foward, incr):
    
    '''Interactive gantt plot (Master schedule)'''
    df = pd.DataFrame(df)
    date = Timestamp(ref_date) + np.timedelta64(incr,"D")
    min_date, max_date = df["start"].min(), df["end"].max()
    start_date = max(date - np.timedelta64(backward,"D"), min_date).date()
    end_date   = min(date + np.timedelta64(foward  ,"D"), max_date).date()
    print(start_date)
    print(end_date)

    X = create_schedule(df.copy(), ref_date, holidays=holidays)
    X = X.sort_values(by="start", ascending=False).reset_index(drop=True)
    
    colors = ["#1B9CFC","#55E6C1","#FC427B","#82589F","#FEA47F"]
    ax = gantt_plot(X, ref_date=ref_date, holidays=holidays, 
                    show_delta=show_delta, colors=colors, 
                    start_date=start_date, end_date=end_date)
    legend = ax.get_legend()
    legend.set_bbox_to_anchor([1.1,0.8], transform=ax.transAxes)

kwargs = dict(df=fixed(data["schedule"]), 
              ref_date=fixed(date.today()), 
              show_delta=[False, True], 
              holidays=fixed(data["PublicHolidays"]), 
              backward=IntSlider(min=1, max=30, step=2, value=15), 
              foward=IntSlider(min=1, max=30, step=2, value=15),
              incr=IntSlider(min=-30, max=30, step=2, value=0))

## Project Schedule

In [5]:
def interactive_project(data, task, ref_date, holidays, 
                        show_delta, backward, foward, incr):
    
    a = pd.DataFrame(data["schedule"])
    a = a.loc[a["task"]==task]
    ref_id = a["ref_id"].values[0]

    try:
        print("Project: ",a["task"].values[0])
        print("ref_id : ",a["ref_id"].values[0])
        print("== Description ==")
        for s in str(a["description"].values[0]).split("\n"):
            print(s)
    except: pass

    if ref_id in list(data.keys()):
        df = pd.DataFrame(data[a["ref_id"].values[0]])   
        df = create_schedule(df, ref_date, holidays)
        df = df.sort_index(ascending=False).reset_index(drop=True)

        date = Timestamp(ref_date) + np.timedelta64(incr,"D")
        min_date, max_date = df["start"].min(), df["end"].max()
        start_date = max(date - np.timedelta64(backward,"D"), min_date)
        end_date   = min(date + np.timedelta64(foward  ,"D"), max_date)

        colors = ["#1B9CFC","#55E6C1","#FC427B","#82589F","#FEA47F"]
        ax = gantt_plot(df, ref_date=ref_date, holidays=holidays, 
                        show_delta=show_delta, colors=colors, 
                        start_date=start_date, end_date=end_date)
        legend = ax.get_legend()
        legend.set_bbox_to_anchor([1.1,0.8], transform=ax.transAxes)
    else:
        print("\n")
        print("< Project details cannot be found. >")

kwargs = dict(data=fixed(data), 
              task=list(data["schedule"]["task"].values()),
              ref_date=fixed(date.today()), 
              show_delta=[False, True], 
              holidays=fixed(data["PublicHolidays"]), 
              backward=IntSlider(min=1, max=100, step=2, value=50), 
              foward=IntSlider(min=1, max=100, step=2, value=50),
              incr=IntSlider(min=-30, max=30, step=2, value=0), 
              xxx=Button(description="Prev Holiday"))

In [6]:
ax = interact(interactive_project, **kwargs)

interactive(children=(Dropdown(description='task', options=('DPD Monitoring Report rev3', 'Pattern D001 improv…